In [ ]:
from helper import aoc_timer
import pandas as pd
import re
import itertools

In [ ]:
class Shop:
    """Class for item shop."""

    def __init__(self, path):
        self.inventory = self.get_shop(path)

    def get_shop(self, path):
        """Read shop inventory into pandas DataFrame."""
        df = []
        for line in open(path).read().split('\n'):
            if not line:
                continue
            if ':' in line:
                # New item type (plus headers)
                idx = line.find(':')
                item_type = line[:idx]
                headers = ['Type', 'Item'] + line[idx+1:].split()
            else:
                # Item row - use RegEx to split on 2+ whitespaces
                df.append({k: v for k, v in zip(headers, [item_type] + re.split(r'\s{2,}', line))})
        return pd.DataFrame(df).astype({k: 'int8' for k in headers[2:]})

    
class Character:
    """Class for RPG characters."""

    def __init__(self, hp=0, damage=0, armor=0, path=None):
        self.path = path  # Optionally set character attributes from input file
        if path is None:
            # Manually set attributes when initializing
            self.hp = hp
            self.damage = damage
            self.armor = armor
        else:
            # Set attributes from input file
            attr = self.get_attr(path)
            self.hp = attr.get('Hit Points', 0)
            self.damage = attr.get('Damage', 0)
            self.armor = attr.get('Armor', 0)
        # Additional attributes
        self.gold = 0
        self.weapon_slots = range(1, 2)  # Must equip a weapon
        self.armor_slots = range(0, 2)   # Armor is optional
        self.ring_slots = range(0, 3)    # Maximum two rings
        self.weapon_equip = None
        self.armor_equip = None
        self.rings_equip = None

    def __str__(self):
        """Print attributes and inventory."""
        attrs = {
            'HP': self.hp,
            'Damage': self.damage,
            'Armor': self.armor
        }
        equip = {
            'Weapon': self.weapon_equip,
            'Armor': self.armor_equip,
            'Rings': self.rings_equip,
            'Cost': self.gold
        }
        return f'Attributes: {attrs}\nInventory: {equip}'

    def get_attr(self, path):
        """Optionally set character attributes from input file."""
        return {k: int(v) for k, v in
                [line.strip().split(': ') for line in open(path).readlines()]}

    def purchase(self, items, shop):
        """Purchase items from the shop, amend attributes, record gold cost."""
        buy = shop.inventory.loc[items]
        self.damage = buy.Damage.sum()
        self.armor = buy.Armor.sum()
        self.gold = buy.Cost.sum()
        # Equip items
        self.weapon_equip = buy.Item.loc[buy.Type == 'Weapons'].tolist()
        self.armor_equip = buy.Item.loc[buy.Type == 'Armmor'].tolist()
        self.rings_equip = buy.Item.loc[buy.Type == 'Rings'].tolist()

    def attack(self, char):
        """Attack given character, char."""
        hit = max(1, self.damage - char.armor)
        char.hp = max(0, char.hp - hit)

    def battle(self, char):
        attacker, defender = self, char
        # Battle to the death
        while self.hp and char.hp:
            attacker.attack(defender)
            attacker, defender = defender, attacker
        # Battle complete
        if self.hp:
            print('You win!')
            print(self)
            return True, self.gold
        else:
            print('You lose!')
            print(self)
            return False, self.gold



In [ ]:
# Itertools testing
me = Character(8, 5, 5)
inv = Shop('shop.txt').inventory

types = inv.Type.unique()
items = zip(
    [me.weapon_slots, me.armor_slots, me.ring_slots],
    [inv.loc[inv.Type == x].index for x in types],
    types
)
cmb = {k: [] for k in types}
for slots, idx, item in items:
    for n in slots:
        for c in itertools.combinations(idx, n):
            cmb[item].append(c)

item_combinations = itertools.product(*cmb.values())
item_combinations = [list(sum(x, ())) for x in item_combinations]  # <-- list of possible item combinations

# Sort combinations by cost ascending and find first one to result in a win
costs = {}
for loadout in item_combinations:
    costs[tuple(loadout)] = inv.Cost.loc[loadout].sum()


In [ ]:
player = Character(100, 0, 0)
boss = Character(path='input.txt')
shop = Shop('shop.txt')

# Part 1: ascending order (x[1]), break on first win
# Part 2: descending order (-x[1]), break on first loss
for items, cost in sorted(costs.items(), key=lambda x: -x[1]):
    player = Character(100, 0, 0)
    boss = Character(path='input.txt')
    player.purchase(list(items), shop)
    result, _ = player.battle(boss)
    if not result:
        print(cost)
        break
